In [3]:
import os

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option("display.float_format", lambda x: f'{x:.3f}')

import warnings
warnings.filterwarnings("ignore")

import numpy as np

In [4]:
def std_col_names(df):
    """
    - Convert feature names to lower case
    """
    df.columns = df.columns.str.lower().str.strip().str.replace(' ', '_')
    return df


In [16]:
# Load DF
df = std_col_names(pd.read_csv('./data/train.csv'))
df.head(5)

,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default
0,CST_115179,ita Bose,46,F,N,Y,0.000,107934.040,612.000,Unknown,1.000,1.000,33070.280,18690.930,73,544.000,2,1,1
1,CST_121920,Alper Jonathan,29,M,N,Y,0.000,109862.620,2771.000,Laborers,2.000,0.000,15329.530,37745.190,52,857.000,0,0,0
2,CST_109330,Umesh Desai,37,M,N,Y,0.000,230153.170,204.000,Laborers,2.000,0.000,48416.600,41598.360,43,650.000,0,0,0
3,CST_128288,Rie,39,F,N,Y,0.000,122325.820,11941.000,Core staff,2.000,0.000,22574.360,32627.760,20,754.000,0,0,0
4,CST_151355,McCool,46,M,Y,Y,0.000,387286.000,1459.000,Core staff,1.000,0.000,38282.950,52950.640,75,927.000,0,0,0


Number of missing values for each feature: 

    owns_car                547  
    no_of_children          774    
    no_of_days_employed     463  
    total_family_members     83  
    migrant_worker           87  
    yearly_debt_payments     95  
    credit_score              8  
    dtype: int64

In [17]:
train_df = df.copy()

In [18]:
init_len = train_df.shape[0]

train_df.dropna(subset=['yearly_debt_payments', 'credit_score'], inplace= True)
post_len = train_df.shape[0]

print(
f'Initial number of records: {init_len}\nNumber of records after dropping:{post_len}'
)

Initial number of records: 45528
Number of records after dropping:45425


In [19]:
train_df['owns_car'].value_counts()

N    29669
Y    15209
Name: owns_car, dtype: int64

In [26]:
train_df['owns_car'].fillna("N", inplace= True)
train_df.fillna(0, inplace= True)

In [29]:
no_null_check = train_df.columns[train_df.isnull().any()].to_list() == []
assert no_null_check == True

In [30]:
default_df = train_df[train_df['credit_card_default'] == 1]

credlim_occ = default_df[['occupation_type', 'credit_limit_used(%)']].groupby(by= 'occupation_type').min().sort_values(by= 'credit_limit_used(%)',ascending= False).rename(columns= {'credit_limit_used(%)': 'occ_credlim'})

credscore_occ = default_df[['occupation_type', 'credit_score']].groupby(by= 'occupation_type').max().sort_values(by= 'credit_score').rename(columns= {'credit_score': 'occ_credscore'})   

In [31]:
train_df = pd.merge(train_df, credlim_occ, how= 'outer', on= 'occupation_type')
train_df = pd.merge(train_df, credscore_occ, how= 'outer', on= 'occupation_type')

In [32]:
train_df.sample(5, random_state=42)

,customer_id,name,age,gender,owns_car,owns_house,no_of_children,net_yearly_income,no_of_days_employed,occupation_type,total_family_members,migrant_worker,yearly_debt_payments,credit_limit,credit_limit_used(%),credit_score,prev_defaults,default_in_last_6months,credit_card_default,occ_credlim,occ_credscore
16256,CST_144540,Sam,37,M,N,N,0.000,98724.500,1296.000,Laborers,2.000,0.000,24303.250,12624.240,33,908.000,0,0,0,70,699.000
42371,CST_142468,Richard,36,M,Y,Y,0.000,184122.830,1419.000,Security staff,2.000,1.000,35713.340,20781.940,29,856.000,0,0,0,71,699.000
24659,CST_116396,Lawrence Hurley,23,F,N,Y,0.000,251246.290,1283.000,Core staff,1.000,0.000,25878.500,72383.220,58,783.000,0,0,0,70,699.000
43684,CST_151340,Blenkinsop,37,F,N,Y,1.000,115582.460,119.000,Cooking staff,3.000,0.000,34318.880,14142.470,34,782.000,0,0,0,71,697.000
24751,CST_120011,Elizabeth Pineau,24,M,Y,Y,0.000,244530.350,6913.000,Core staff,2.000,0.000,8790.600,32145.950,56,861.000,0,0,0,70,699.000


In [33]:
min_credlim_default = round(default_df['credit_limit_used(%)'].min(), 2)
print(f'Minimum credit limit at which customer has defaulted is : {min_credlim_default}')

max_credscore_default = round(default_df['credit_score'].max(), 2)
print(f'Max credit score for which customer has defaulted is : {max_credscore_default}')

Minimum credit limit at which customer has defaulted is : 70
Max credit score for which customer has defaulted is : 699.0


In [34]:
train_df['above_min_credlim_def'] = (train_df['credit_limit_used(%)']>=min_credlim_default).astype('int')
train_df['below_min_credscore_def'] = (train_df['credit_score']<=max_credscore_default).astype('int')

In [35]:
train_df['above_min_credlim_occ'] = (train_df['credit_limit_used(%)']>=train_df['occ_credlim']).astype('int')
train_df['below_min_credscore_occ'] = (train_df['credit_score']<=train_df['occ_credscore']).astype('int')

In [36]:
train_df['months_employed'] = train_df['no_of_days_employed'] / 30

train_df['credlim_to_income'] =train_df['credit_limit'] / train_df['net_yearly_income'] 

train_df['debt_to_income'] = train_df['yearly_debt_payments'] / train_df['net_yearly_income']


In [37]:
train_df['num_gender'] = train_df['gender'].map({'XNA':-1, 'F': 0, 'M': 1})

mapper_yn = {'Y': 1, 'N': 0}
train_df['num_car'] = train_df['owns_car'].map(mapper_yn, na_action= 'ignore')
train_df['num_house'] = train_df['owns_house'].map(mapper_yn)

In [39]:
train_df.columns

Index(['customer_id', 'name', 'age', 'gender', 'owns_car', 'owns_house',
       'no_of_children', 'net_yearly_income', 'no_of_days_employed',
       'occupation_type', 'total_family_members', 'migrant_worker',
       'yearly_debt_payments', 'credit_limit', 'credit_limit_used(%)',
       'credit_score', 'prev_defaults', 'default_in_last_6months',
       'credit_card_default', 'occ_credlim', 'occ_credscore',
       'above_min_credlim_def', 'below_min_credscore_def',
       'above_min_credlim_occ', 'below_min_credscore_occ', 'months_employed',
       'credlim_to_income', 'debt_to_income', 'num_gender', 'num_car',
       'num_house'],
      dtype='object')

In [40]:
cols = [
  'age', 'num_gender', 'num_car', 'num_house',
       'no_of_children', 'total_family_members', 'migrant_worker', 
    'prev_defaults', 'default_in_last_6months',
    'above_min_credlim_def', 'above_min_credlim_occ', 'below_min_credscore_def', 'below_min_credscore_occ', 'months_employed',
       'credlim_to_income', 'debt_to_income'
]

target = 'credit_card_default'

In [41]:
print(f'Number of features: {len(cols)}')

Number of features: 16


In [48]:
from sklearn.feature_selection import mutual_info_classif

mi = mutual_info_classif(train_df[cols], train_df[target], n_neighbors= 5, random_state= 42, discrete_features= True)
mi_df = pd.DataFrame(mi, index= cols, columns=['mutual_info']).reset_index().sort_values(by='mutual_info', ascending= False)


In [50]:
mi_df[mi_df['mutual_info'] >0]['index'].to_list()

['credlim_to_income',
 'debt_to_income',
 'prev_defaults',
 'default_in_last_6months',
 'below_min_credscore_occ',
 'below_min_credscore_def',
 'above_min_credlim_occ',
 'above_min_credlim_def',
 'months_employed',
 'num_gender',
 'migrant_worker',
 'no_of_children',
 'total_family_members',
 'age',
 'num_car',
 'num_house']